<a href="https://colab.research.google.com/github/Ditager55/Proyecto-Final---Data-3/blob/main/Proyecto_Final_Data3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Capstone Project - La Batalla de los Vecindarios (Week 2)
### Applied Data Science Capstone by IBM/Coursera

# **PLANTEAMIENTO DEL PROBLEMA**

# **Introducción**

Debido al incremento de la demanda de departamentos en Toronto, por la migración, crecimiento poblacional e independización de jovenes canadiences. Las personas se han visto en la obligación de buscar un lugar donde vivir acorde a sus ingresos y necesidades. Se debe tener en cuenta que los departamentos en esta ciudad varian en precio según su ubicación, cercanía a transporte, cercarnía al areopuerto, centros comerciales y principales vias de acceso. Además, debemos tener en cuenta que Tornto cuenta con 3 areopuertos ubicados en puntos estratégicos de la ciudad. Por eso se plantea el siguiente problema comercial.

# **Problema Comercial**

Los nuevos residentes y residentes actuales que en el futuro deseen trasladarse a un lugar más tranquilo que la ciudad tienen para elequipo de ciencia de datos la siguiente interrogante: **¿Cuáles son los mejores distritos para vivir en la ciudad de Toronto?**

# **Objetivo**

El objetivo principal de la presente investigación es identificar los mejores condominios para que quienes estén interesados logren comprar una propiedad adecuada a sus necesidades, para esto analizaremos los siguientes factores: 


*   Precio del departamento
*   Seguridad de la zona

*   Si las zonas comerciales están cerca
*   Si hay colegios, hospitales cercanos

# **Metodologia**

1.   Con los datos de Foursquare de Toronto se identificarán los distritos que existen en cada zona etiquetada por el código postal.

2.   Se podran observar todos los distritos, para buscar departamentos y colocarlos en un tabla, con las caracteristicas que contiene.
3.   Se identificarán todos los departamentos disponibles por colores según distritos.
4.   Finalmente, se seleccionara los departamentos que cumplan con la mayoria
de los criterios .


# **Datos**


Lo datos que se usarán en el presentte proyecto han sido brindados por Foursquare, por todo el repositorio de locales con su ubicación que permiten determinar la conveniencia geográfica de la puesta en marcha de buscar departamentos de una determinada zona.
Además, se han utilizado fuentes confiables de información para guiar la bpusqueda de datos útiles para realizar la parte de resultados y discusión.
 
Link: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

# **Nootebok para Proyecto Final**

In [1]:
import pandas as pd
import numpy as np
print ("¡Hola, Proyecto Final del Curso!")

¡Hola, Proyecto Final del Curso!


In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import requests
!pip install geocoder
import geocoder

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 98 kB 2.1 MB/s 


- Se importan los codigos postales desde el link de Wikipedia proporcionado por la platafroma, debido a problemas con Foursquare.

In [3]:
linkpost= "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

- Se genera una tabla que contiene los codigos postales, el distrito al que pertenecen y los vecindarios correspondientes para identificarlos dentro del mapa de ciudad que se visualizará más adelante.

In [4]:
page = requests.get(linkpost)
soup = BeautifulSoup(page.content, 'html.parser')

table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# Imprimir (table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

Se muestra los 11 primeros registros de codigos postales, distritos y vecindarios que serviran a los futuros compradores para elegir su residencia ideal. 

In [5]:
df.head(11)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


- Con la función "geo" obtendremos las cordenadas de latitud y longitud de los distritos por codigo postal. Se usa la función head para visualizar la tabla. 

In [6]:
geo_url = "https://cocl.us/Geospatial_data"
geo_df = pd.read_csv(geo_url)
geo_df.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
geo_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


- Se aplica la función merge para trabajar los codigos postales y coordenadas de los vecindarios/condominios y junto a la función head, se visualizan los 5 primeros registros. 

In [7]:
df = pd.merge(df, geo_df, on='PostalCode')
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


- Con la función "groupby" y "count" se agrupan los condominios según su distrito al que pertenecen y se contabilizan para etiquetarlos en el mapa de la ciudad líneas más abajo. 

In [8]:
df.groupby('Borough').count()['Neighborhood']

Borough
Central Toronto            9
Downtown Toronto          17
Downtown Toronto Stn A     1
East Toronto               4
East Toronto Business      1
East York                  4
East York/East Toronto     1
Etobicoke                 11
Etobicoke Northwest        1
Mississauga                1
North York                24
Queen's Park               1
Scarborough               17
West Toronto               6
York                       5
Name: Neighborhood, dtype: int64

- Se crea el data frame toronto donde las columnas son el codigo postal, el distrito, el condominio, la latitud y la longitud que son necesarias para crear un mapa donde se identifiquen las propiedades disponibles para analizar.

In [9]:
df_toronto = df
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [10]:
boroughs = df_toronto['Borough'].unique().tolist()

Se delimitan las coordenadas de la ciudad de Toronto...

---



In [11]:
lat_toronto = df_toronto['Latitude'].mean()
lon_toronto = df_toronto['Longitude'].mean()
print('Las coordenadas geográficas de Toronto son {}, {}'.format(lat_toronto, lon_toronto))

Las coordenadas geográficas de Toronto son 43.704607733980595, -79.39715291165048


In [12]:
borough_color = {}
for borough in boroughs:
    borough_color[borough]= '#%02X%02X%02X' % tuple(np.random.choice(range(256), size=3))

- Se importa la libreria "folium"

In [13]:
import folium

. Aplicando la función "map" de la libreria "folium" y usando la latitud y longitud de la ciudad de Toronto y las ubicaciones de los condominión, se realiza un mapa a escala de 10.5.
-Además, mediante un circulo de color diferenciado se etiquetará los condominios disponibles para venta futura, según el distrito donde se encuentre. 

In [14]:
map_toronto = folium.Map(location=[lat_toronto, lon_toronto], zoom_start=10.5)

for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], 
                                           df_toronto['Longitude'],
                                           df_toronto['Borough'], 
                                           df_toronto['Neighborhood']):
    label_text = borough + ' - ' + neighborhood
    label = folium.Popup(label_text)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=borough_color[borough],
        fill_color=borough_color[borough],
        fill_opacity=0.8).add_to(map_toronto)  
    
map_toronto

# **Conclusiones/Discusion**

**Conclusiones**
1. Existen 15 distritos en Toronto donde actualmente tienen disponibles condominios con departamentos a la venta. 
2. Para las personas que buscan un departamento cerca al mar existen cuatro distritos disponibles para comprar cerca al centro de Toronto: Scarborough, East Toronto,  Downtown Toronto, West Toronto y Etobicoke - New Toronto. Serán sus necesidades particulares que los hagan decidir dónde comprar su nueva residencia. Por ejemplo: Si trabajan en el centro financiero el Downtown de Toronto es ideal para vivir, aunque es un lugar de precio elevado. Por otro lado, si tu opción de residencia es una alejada de la ciudad, pero estar cerca a las principales arterias de acceso a la ciudad lo tuyo será North York - Lawrence Manor, Lawrence Heights. 

**Discusión**
1. Hubiese sido ideal tener un registro del nivel de delincuencia de cada zona para segmentarlas mejor y discriminar entre distritos y condominios.
2. La información solo nos da una información parcial de qué puede buscar el futuro residente en Toronto pues no tenemos información de las familias o sus caracteristicas para hacer conjeturas más profundas.

#**Resultados**

- Se logró identificar los condominios segmentados por distritos de la ciudad de Toronto, esto cumplió con el objetivo principal de la investigación presentada. 
- Se utilizaron librerías proporcionadas por el curso de Data Science y se obtuvieron resultados favorables en la aplicación de códigos específicos. 